# Playing with data

read data from csv

In [ ]:
import pandas as pd
df_season = pd.read_csv("static/data/NBA_season.csv")
df_season = df_season[df_season.columns.tolist()[1:]]
df_playoffs = pd.read_csv("static/data/NBA_playoffs.csv")
df_playoffs= df_playoffs[df_playoffs.columns.tolist()[1:]]
df_teams = pd.read_csv("static/data/Teams.csv")
df_teams = df_teams.set_index("Unnamed: 0")
df_teams.index.name = "Team"

add team win rate to df_season

In [28]:
df_season = df_season[(df_season['G'] >= 40) & (df_season['Year'] >= 1978) & (df_season['TmWin%'] >= 48)]
df_playoffs = df_playoffs[(df_playoffs['G'] >= 3) & (df_playoffs['Year'] >= 1978)]

## Calculate average stats by pos

In [9]:
group = df_season[(df_season['Year'] >= 1980) & (df_season['MP'] >= 20)].groupby('Pos')

df_avg_stats_by_pos_mixed = group[["AST", "PTS", "TOV", "BLK", "STL", "ORB", "DRB"]].mean()
df_avg_stats_by_pos = df_avg_stats_by_pos_mixed.loc[['PF', 'C', 'PG', 'SF', 'SG']]
df_count_by_pos = group['id'].count()

In [10]:
count_by_pos = {
    'C': df_count_by_pos.loc['C'],
    'PF': df_count_by_pos.loc['PF'],
    'PG': df_count_by_pos.loc['PG'],
    'SF': df_count_by_pos.loc['SF'],
    'SG': df_count_by_pos.loc['SG'],
}

for pos, row in df_avg_stats_by_pos_mixed.iterrows():
    if pos not in ['PF', 'C', 'PG', 'SF', 'SG']:
        # get amount of players with this mixed position
        amount_mixed = df_count_by_pos.loc[pos]

        # get the sum of the mixed stats for each stat
        stats = []
        for stat in row:
            stats.append(stat * amount_mixed)

        # for each position, add avg to position
        for p in pos.split(','):
            p = p.strip()
            amount_pure = count_by_pos[p]

            # get average stats for this pure pos
            pos_row = df_avg_stats_by_pos.loc[p]
            for i in range(len(stats)):
                # create new weighted arithmetic mean
                new_stat = (pos_row[i] * amount_pure + stats[i]) / (amount_pure + amount_mixed)
                pos_row[i] = new_stat

            # update amount for this pos
            count_by_pos[p] += amount_mixed

In [11]:
df_avg_stats_by_pos

,AST,PTS,TOV,BLK,STL,ORB,DRB
Pos,,,,,,,
PF,1.949051,13.090512,1.768501,0.792694,0.848292,2.149146,5.082638
C,1.637459,12.275679,1.771553,1.434745,0.707383,2.469707,5.542997
PG,5.697905,12.883524,2.184571,0.198571,1.284857,0.674095,2.477524
SF,2.516463,14.219003,1.799059,0.504610,1.034807,1.398589,3.579210
SG,3.156130,14.305517,1.815937,0.308581,1.099650,0.859370,2.641856


## Calculate player score for season and playoffs

option 1 - best year of the player

In [56]:
def get_best_year(df):
    ser = df.groupby(['id'])['score2'].max()
    d = {}
    for key, value in ser.iteritems():
        entry = df[(df['id'] == key) & (df['score2'] == value)].iloc[0]
        d[key] = [entry['Name'], entry['Year'], entry['score2']]

    return pd.DataFrame.from_dict(d, orient="index", columns=['Name', 'Year', 'score2']).sort_values(by="score2", ascending=False)

,id,Name,From,To,Height,Weight,Year,Age,Tm,Lg,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,TmWin%,score2
0,abdelal01,Alaa Abdelnaby,1991,1995,208.28,108.862169,1991,22.0,POR,NBA,...,1.4,2.1,0.3,0.1,0.3,0.5,0.9,3.1,53.6,2.651714
1,abdelal01,Alaa Abdelnaby,1991,1995,208.28,108.862169,1992,23.0,POR,NBA,...,2.5,3.7,0.4,0.4,0.2,0.9,1.9,6.1,53.6,4.851042
7,abernto01,Tom Abernethy,1977,1981,200.66,99.790321,1978,23.0,LAL,NBA,...,2.2,3.6,1.4,0.8,0.3,0.7,1.7,6.8,59.4,6.643703
8,abernto01,Tom Abernethy,1977,1981,200.66,99.790321,1979,24.0,GSW,NBA,...,2.0,3.1,1.1,0.6,0.2,0.5,1.9,6.0,48.5,5.661622
9,abernto01,Tom Abernethy,1977,1981,200.66,99.790321,1980,25.0,GSW,NBA,...,1.9,2.9,1.3,0.5,0.2,0.6,1.8,5.4,48.5,5.112979
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22972,zellety01,Tyler Zeller,2013,2020,213.36,114.758870,2017,27.0,BOS,NBA,...,1.6,2.4,0.8,0.1,0.4,0.4,1.2,3.5,58.9,3.330495
22977,zipsepa01,Paul Zipser,2017,2018,203.20,97.522360,2017,22.0,CHI,NBA,...,2.5,2.8,0.8,0.3,0.4,0.9,1.8,5.5,51.1,4.035308
22978,zipsepa01,Paul Zipser,2017,2018,203.20,97.522360,2018,23.0,CHI,NBA,...,2.2,2.4,0.9,0.4,0.3,0.8,1.6,4.0,51.1,3.150889
22981,zhizhwa01,Wang Zhizhi,2001,2005,213.36,115.666054,2002,24.0,DAL,NBA,...,1.7,2.0,0.4,0.2,0.3,0.5,1.5,5.6,50.4,3.809139


option 2 - average of last max 4 years

In [43]:
rows_season = {}
rows_playoffs = {}

for player in df_season['id'].unique():
    df_player_season = df_season[df_season['id'] == player].sort_values(by='score2', ascending=False).head(4)
    df_player_playoffs = df_playoffs[df_playoffs['id'] == player].sort_values(by='score2', ascending=False).head(4)
    avg_season = df_player_season['score2'].mean()
    avg_playoffs = df_player_playoffs['score2'].mean()

    rows_season[player] = [df_player_season.iloc[0]['Name'], list(df_player_season['Year'].sort_values()), avg_season]
    try:
        rows_playoffs[player] = [df_player_playoffs.iloc[0]['Name'], list(df_player_playoffs['Year'].sort_values()), avg_playoffs]
    except IndexError:
        pass

df_best4_season = pd.DataFrame().from_dict(rows_season, orient='index', columns=['Name', 'Years', 'score2'])
df_best4_season.index.name = "id"
df_best4_playoffs = pd.DataFrame().from_dict(rows_playoffs, orient='index', columns=['Name', 'Years', 'score2'])
df_best4_playoffs.index.name = "id"

In [44]:
df_best4_season.sort_values(by='score2', ascending=False)

,Name,Years,score2
id,,,
nealll01,Lloyd Neal,[1978],33.258842
jordami01,Michael Jordan,"[1987, 1988, 1989, 1990]",29.734949
hardeja01,James Harden,"[2017, 2018, 2019, 2020]",28.554065
perrycu01,Curtis Perry,[1978],28.505449
johnsma02,Magic Johnson,"[1987, 1989, 1990, 1991]",28.108707
...,...,...,...
kornefr01,Frank Kornet,[1990],1.666161
hayesst01,Steve Hayes,"[1984, 1986]",1.663523
lewisra01,Ralph Lewis,[1988],1.522468


In [45]:
df_best4_playoffs.sort_values(by='score2', ascending=False)

,Name,Years,score2
id,,,
jordami01,Michael Jordan,"[1986, 1989, 1990, 1991]",31.431289
jamesle01,LeBron James,"[2009, 2017, 2018, 2020]",30.090656
johnsma02,Magic Johnson,"[1985, 1986, 1987, 1990]",29.743493
doncilu01,Luka Dončić,"[2020, 2021]",28.411557
stockjo01,John Stockton,"[1988, 1989, 1990, 1991]",28.329942
...,...,...,...
ntilila01,Frank Ntilikina,[2021],0.538628
higgise01,Sean Higgins,[1991],0.538628
newbiiv01,Ivano Newbill,[1997],0.484641
